In [1]:
import pandas as pd
import numpy as np



In [2]:

df2 = pd.read_csv('imdb_reviews.csv')
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25000 entries, 0 to 24999
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Reviews    25000 non-null  object
 1   Sentiment  25000 non-null  object
dtypes: object(2)
memory usage: 390.8+ KB


In [4]:
import nltk
import string
import os

In [5]:
import re
import string
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer



In [6]:
stop = set(stopwords.words('english')) 
sno = nltk.stem.SnowballStemmer('english') 

def cleanhtml(sentence): 
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, ' ', sentence)
    return cleantext
def cleanpunc(sentence): 
    cleaned = re.sub(r'[?|!|\'|"|#]',r'',sentence)
    cleaned = re.sub(r'[.|,|)|(|\|/]',r' ',cleaned)
    return  cleaned

In [31]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to C:\Users\Mostafa
[nltk_data]     Hesham\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [33]:
data = df2

In [34]:
data['Sentiment']

0        positive
1        negative
2        negative
3        positive
4        negative
           ...   
24995    positive
24996    negative
24997    negative
24998    positive
24999    negative
Name: Sentiment, Length: 25000, dtype: object

In [35]:
i=0
str1=' '
final_string=[]
all_positive_words=[] 
all_negative_words=[] 
s=''
for sent in data['Reviews'].values:
    filtered_sentence=[]
    sent=cleanhtml(sent) 
    for w in sent.split():
        for cleaned_words in cleanpunc(w).split():
            if((cleaned_words.isalpha()) & (len(cleaned_words)>2)):    
                if(cleaned_words.lower() not in stop):
                    s=(sno.stem(cleaned_words.lower())).encode('utf8')
                    filtered_sentence.append(s)
                    if (data['Sentiment'].values)[i] == "positive": 
                        all_positive_words.append(s) 
                    if(data['Sentiment'].values)[i] == "negative":
                        all_negative_words.append(s) 
                else:
                    continue
            else:
                continue 
    
    str1 = b" ".join(filtered_sentence) 
    
    final_string.append(str1)
    i+=1

In [36]:
data['cleaned_review']=final_string

In [40]:
data.head()

,Reviews,Sentiment,cleaned_review
0,<START this film was just brilliant casting lo...,positive,b'amaz actor director father came scottish isl...
1,<START big hair big boobs bad music and a gian...,negative,b'hair big lot boob men wear cut shirt show si...
2,<START this has to be one of the worst films o...,negative,b'work watch feebl excus film must look like g...
3,<START the <UNK> <UNK> at storytelling the tra...,positive,b'storytel tradit sort mani year event still s...
4,<START worst mistake of my life br br i picked...,negative,b'watch burn felt better anyth els ive ever do...


In [39]:
data['cleaned_review'][150]

b'unawar want enjoy countri littl know would happen next photographi set realist natur day wes craven disturb film fact place like still exist america countri folk still citi peopl almost centuri half sinc civil war enjoy film film rural section south georgia still exist dont drive past mobil area still sinc'

In [80]:
def posneg(x):
    if x=="negative":
        return 0
    elif x=="positive":
        return 1
    return x

In [81]:
data['Sentiment'] = data['Sentiment'].apply(posneg)

In [89]:
data.iloc[0]

Reviews           <START this film was just brilliant casting lo...
Sentiment                                                         1
cleaned_review    b'amaz actor director father came scottish isl...
Name: 0, dtype: object

In [92]:
data['cleaned_review'][0]

b'amaz actor director father came scottish island love fact real connect film witti remark throughout film great brilliant much bought film soon releas would recommend everyon watch fli fish amaz realli cri end sad know say cri film must good definit also two littl boy play norman paul brilliant children often left list think star play grown big profil whole film children amaz prais done dont think whole stori love true someon life share'

In [93]:
x = data['cleaned_review']
y = data['Sentiment']

In [94]:
X_train, X_test, y_train, y_test = train_test_split(x,y,test_size=0.2,random_state=8)

In [95]:
x

0        b'amaz actor director father came scottish isl...
1        b'hair big lot boob men wear cut shirt show si...
2        b'work watch feebl excus film must look like g...
3        b'storytel tradit sort mani year event still s...
4        b'watch burn felt better anyth els ive ever do...
                               ...                        
24995    b'focus dad relax peac thing go still home gra...
24996    b'make look dead steal bodi exact sure bodi do...
24997    b'nativ peopl worship cat dog peopl war upon l...
24998    b'would back earli happen friend realli surpri...
24999    b'set set rob style remak mood never instead l...
Name: cleaned_review, Length: 25000, dtype: object

In [96]:
vectorizer = TfidfVectorizer(stop_words='english')

In [75]:
vectorizer.fit(x)

vectorizer.vocabulary_

{'cleaned_review': 0}

In [7]:

from sklearn.feature_extraction.text import TfidfVectorizer

In [97]:
vectorizer = TfidfVectorizer(stop_words='english')
vectorizer.fit(X_train)

TfidfVectorizer(stop_words='english')

In [99]:
tf_idf_vect = TfidfVectorizer()
tf_idf_train = tf_idf_vect.fit_transform(X_train.values)
tf_idf_test = tf_idf_vect.transform(X_test.values)

In [100]:
alpha_range = list(np.arange(1,50,5))
len(alpha_range)

10

In [102]:
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score

In [103]:
from sklearn.naive_bayes import MultinomialNB

alpha_scores=[]

for a in alpha_range:
    clf = MultinomialNB(alpha=a)
    scores = cross_val_score(clf, tf_idf_train, y_train, cv=5, scoring='accuracy')
    alpha_scores.append(scores.mean())
    print(a,scores.mean())

1 0.8397500000000001
6 0.83985
11 0.8343499999999999
16 0.83095
21 0.8269500000000001
26 0.8247
31 0.8218
36 0.8186499999999999
41 0.81645
46 0.8145


In [104]:
clf = MultinomialNB(alpha=6)
clf.fit(tf_idf_train,y_train)

MultinomialNB(alpha=6)

In [105]:
y_pred_test = clf.predict(tf_idf_test)

In [106]:
acc = accuracy_score(y_test, y_pred_test, normalize=True) * float(100)
print('\n >>> Test accuracy is',(acc))


 >>> Test accuracy is 84.08
